In [1]:
# INTRODUCTION: THE PROBLEM SETTING

In [2]:
import warnings; warnings.simplefilter('ignore')
from sklearn import datasets
import scipy
from sklearn import svm
import pickle
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix

In [3]:
# import basics datasets
iris = datasets.load_iris()
digits = datasets.load_digits()

In [4]:
# predict a digit
#clf = svm.SVC(gamma=0.001, C=100.)
clf = KNeighborsClassifier(3)
X, y = digits.data[:-1], digits.target[:-1]
clf.fit(X,y)
clf.predict(digits.data[-1:])

array([8])

In [5]:
# predict a string
clf = svm.SVC()
X, y = iris.data, iris.target
clf.fit(X, y)
print(X[0:1])
clf.predict(X[0:1])

[[ 5.1  3.5  1.4  0.2]]


array([0])

In [6]:
iris.name

AttributeError: name

In [ ]:
# save a model
s = pickle.dumps(clf)
clf2 = pickle.loads(s)
clf2.predict(X[0:1])

In [ ]:
# CLASSIFIER COMPARISON FOR DIGITS DATASET

# Code source: Gaël Varoquaux
#              Andreas Müller
# Modified for documentation by Jaques Grobler
# License: BSD 3 clause

# Load estimators/classifiers and Datasets

h = .02  # step size in the mesh

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Decision Tree",
         "Random Forest", "AdaBoost", "Naive Bayes", "Linear Discriminant Analysis",
         "Quadratic Discriminant Analysis"]
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    AdaBoostClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis()]

In [ ]:
# CLASSIFIER COMPARISON

In [ ]:
X, y = make_classification(n_features=2, n_redundant=0, n_informative=2,
                           random_state=1, n_clusters_per_class=1)
rng = np.random.RandomState(2)
X += 2 * rng.uniform(size=X.shape)
linearly_separable = (X, y)

datasets = [make_moons(noise=0.3, random_state=0),
            make_circles(noise=0.2, factor=0.5, random_state=1),
            linearly_separable]

In [ ]:
# CONFUSION MATRIX
def plot_confusion_matrix(cm, x_names, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(x_names))
    plt.xticks(tick_marks, x_names, rotation=45)
    plt.yticks(tick_marks, x_names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
figure = plt.figure(figsize=(27, 9))
i = 1
j = 7
# iterate over datasets
for ds in datasets:
    # preprocess dataset, split into training and test part
    X, y = ds
    
    # normalize data (n_samples) fiting first and then transforming
    X = StandardScaler().fit_transform(X)
    
    # split data (n_samples,n_features) into test (0.4 or 40%) and train data 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4)

    ####################### is "y" extracted from X (n_samples)???? X[:,0] and X[:,1]? where are n_features? Y?
    
    # generate the mesh +/-0.5 
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    # just plot the dataset first
    cm = plt.cm.RdBu
    cm_bright = ListedColormap(['#FF0000', '#0000FF'])
    ax = plt.subplot((len(datasets))*2, len(classifiers) + 1, i)
    
    # Plot the training points
    ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright)
    
    # and testing points (transparent)
    ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright, alpha=0.6)
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xticks(())
    ax.set_yticks(())
    i += 1
    # iterate over classifiers
    for name, clf in zip(names, classifiers):
        ax = plt.subplot((len(datasets))*2, len(classifiers) + 1, i)
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        y_pred = clf.fit(X_train, y_train).predict(X_test)

        # Plot the decision boundary. For that, we will assign a color to each
        # point in the mesh [x_min, m_max]x[y_min, y_max].
        if hasattr(clf, "decision_function"):
            Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
        else:
            Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]

        # Put the result into a color plot
        Z = Z.reshape(xx.shape)
        ax.contourf(xx, yy, Z, cmap=cm, alpha=.8)

        # Plot also the training points
        ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright)
        
        # and testing points
        ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright,
                   alpha=0.6)

        ax.set_xlim(xx.min(), xx.max())
        ax.set_ylim(yy.min(), yy.max())
        ax.set_xticks(())
        ax.set_yticks(())
        ax.set_title(name)
        ax.text(xx.max() - .3, yy.min() + .3, ('%.2f' % score).lstrip('0'),
                size=15, horizontalalignment='right')
        
        ax = plt.subplot((len(datasets))*2, len(classifiers) + 1, j)
        # Compute confusion matrix
        cm = confusion_matrix(y_test, Z)
        #np.set_printoptions(precision=2)
    #   # print('Confusion matrix, without normalization')
       # print(cm)
    #   # plt.figure()
    #    plot_confusion_matrix(cm, X_train[:, 1])
        i += 1
        j += 1

    figure.subplots_adjust(left=.02, right=.98)
plt.show()
plt.close()